In [102]:
from scipy.spatial import cKDTree
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import uproot

## Fiducial check for a single .root file

In [103]:
# Constants for the fiducial cut calculations
scoringPlaneZ = 239.9985
ecalFaceZ = 247.932
cell_radius = 5.0

# Project the recoil coordinates from the scoring plane to the ecal face
def projection(Recoilx, Recoily, Recoilz, RPx, RPy, RPz, HitZ):
    x_final = Recoilx + RPx / RPz * (HitZ - Recoilz) if RPz != 0 else 0
    y_final = Recoily + RPy / RPz * (HitZ - Recoilz) if RPy != 0 else 0
    return (x_final, y_final)

# Calculate the Euclidean distance between two points
def dist(cell, point):
    return np.sqrt((cell[0] - point[0])**2 + (cell[1] - point[1])**2)

# Load cell map from file
def load_cell_map(filepath):
    cellMap = np.genfromtxt(filepath, dtype={'names': ('x', 'y', 'id'), 'formats': ('f4', 'f4', 'i4')}, usecols=[1,2,0])
    print(f"Loaded detector info from {filepath}")
    # Convert structured array to a regular 2D numpy array
    cell_coordinates = np.vstack((cellMap['x'], cellMap['y'])).T
    # Construct KD-Tree with the coordinates array
    cell_tree = cKDTree(cell_coordinates)
    return cell_tree
    return cell_tree

# Apply projection check, oncell for fiducial and offcell for non-fiducial
def check_projection(recoilX, recoilY, recoilZ, recoilPx, recoilPy, recoilPz, recoilElectron, cell_tree):
    valid_indices = np.where(recoilElectron != -1)[0]
    # Compute all projections at once if possible
    projections = np.array([projection(recoilX[idx][recoilElectron[idx]], recoilY[idx][recoilElectron[idx]], recoilZ[idx][recoilElectron[idx]], 
                                        recoilPx[idx][recoilElectron[idx]], recoilPy[idx][recoilElectron[idx]], recoilPz[idx][recoilElectron[idx]], ecalFaceZ) 
                            for idx in valid_indices])
    
    oncell_cut = np.zeros_like(recoilElectron, dtype=bool)

    # Spatial check using KD-Tree
    for idx, proj in zip(valid_indices, projections):
        if cell_tree.query_ball_point(proj, cell_radius):
            oncell_cut[idx] = True

    # Output results and return the cut array
    oncell_events = np.sum(oncell_cut)
    offcell_events = len(valid_indices) - oncell_events
    off_on_ratio = offcell_events / oncell_events if oncell_events > 0 else float('inf')
    
    print(f'oncell recoil electron: {oncell_events}')
    print(f'offcell recoil electron: {offcell_events}')
    print(f'offcell to oncell ratio: {off_on_ratio}')
    
    return oncell_cut

In [104]:
# Skim target scoring plane data for valid events, where recoil electrons exist
def skim_tsp_events(pdgID, z, pz):
    r_electron = np.full(len(pdgID), -1, dtype=int)
    no_electron = 0
    no_valid_z = 0
    no_valid_pz = 0
    # Loop through all events for recoil check
    for i in range(len(pdgID)):
        # Find indices where particle ID is 11 (electron)
        electron_indices = np.where(pdgID[i] == 11)[0]
        # Skip electron-less events
        if len(electron_indices) == 0:
            no_electron += 1
            continue
        # Find electron hits on interested position range
        valid_tsp_z = []
        for j in electron_indices:
            if z[i][j]>0.1 and z[i][j]<0.2:
                valid_tsp_z.append(j)
        # Skip out-range events
        if len(valid_tsp_z) == 0:
            no_valid_z += 1
            continue
        # Find recoil electron, which has maximum pz among all electrons in the event
        pz_max = valid_tsp_z[0]
        for k in valid_tsp_z:
            if pz[i][k]>pz[i][pz_max]:
                pz_max = k
        # Skip irregular events
        if pz[i][pz_max] == 0 or pz[i][pz_max] == -9999:
            no_valid_pz += 1
            continue
        r_electron[i] = pz_max
    # Print results and return the recoil array
    total_events = len(pdgID)
    valid_events = len(np.where(r_electron != -1)[0])
    num_validation = (no_electron+no_valid_z+no_valid_pz+valid_events) == len(pdgID)
    print('total events: {}'.format(total_events))
    print('- no-electron events: {}'.format(no_electron))
    print('- invalid-z events: {}'.format(no_valid_z))
    print('- invalid-pz events: {}'.format(no_valid_pz))
    print('+ valid events: {}'.format(valid_events))
    print('validation: {}'.format(num_validation))
    return r_electron

In [105]:
# Skim ecal scoring plane data for valid events, where recoil electrons exist
def skim_esp_events(pdgID, z, pz):
    r_electron = np.full(len(pdgID), -1, dtype=int)
    no_electron = 0
    no_valid_z = 0
    no_valid_pz = 0
    # Loop through all events for recoil check
    for i in range(len(pdgID)):
        # Find indices where particle ID is 11 (electron)
        electron_indices = np.where(pdgID[i] == 11)[0]
        # Skip electron-less events
        if len(electron_indices) == 0:
            no_electron += 1
            continue
        # Find electron hits on interested position range
        valid_esp_z = []
        for j in electron_indices:
            if z[i][j]>239 and z[i][j]<240:
                valid_esp_z.append(j)
        # Skip out-range events
        if len(valid_esp_z) == 0:
            no_valid_z += 1
            continue
        # Find recoil electron, which has maximum pz among all electrons in the event
        pz_max = valid_esp_z[0]
        for k in valid_esp_z:
            if pz[i][k]>pz[i][pz_max]:
                pz_max = k
        # Skip irregular events
        if pz[i][pz_max] == 0 or pz[i][pz_max] == -9999:
            no_valid_pz += 1
            continue
        r_electron[i] = pz_max
    # Print results and return the recoil array
    total_events = len(pdgID)
    valid_events = len(np.where(r_electron != -1)[0])
    num_validation = (no_electron+no_valid_z+no_valid_pz+valid_events) == len(pdgID)
    print('total events: {}'.format(total_events))
    print('- no-electron events: {}'.format(no_electron))
    print('- invalid-z events: {}'.format(no_valid_z))
    print('- invalid-pz events: {}'.format(no_valid_pz))
    print('+ valid events: {}'.format(valid_events))
    print('validation: {}'.format(num_validation))
    return r_electron

### Load leaves

In [106]:
# Load cell information
cells = load_cell_map('/home/aether_zhou_2023/LDMX-scripts/GraphNet/data/v14/cellmodule.txt')

# Path to the ROOT file
# file_path = '/home/vamitamas/NonFiducialSimu/events_nonfiducial_fullEcal_production.root'
# file_path = '/mnt/d/Repositories/rootfiles/Processed8GeV/v14_8gev_0.1_XCal_total_40.root'
# file_path = '/mnt/d/Repositories/rootfiles/Samples8GeV/Ap0.001GeV_sim/mc_v14-8gev-8.0GeV-1e-signal_W_noDecay_sim_run10059_t1699049052.root'
# file_path = '/home/duncansw/GraphNet_input/v14/8gev/v3_tskim/XCal_total/v14_8gev_0.1_XCal_total_4.root'
file_path = '/home/vamitamas/Samples8GeV/Ap1GeV_sim/mc_v14-8gev-8.0GeV-1e-signal_W_noDecay_sim_run10000227_t1699048086.root'

Loaded detector info from /home/aether_zhou_2023/LDMX-scripts/GraphNet/data/v14/cellmodule.txt


In [107]:
# Open the ROOT file and load branches
with uproot.open(file_path)['LDMX_Events'] as tree:
    esp_leaf_pdgID = tree['EcalScoringPlaneHits_signal.pdgID_'].array(library='np')
    esp_leaf_x = tree['EcalScoringPlaneHits_signal.x_'].array(library='np')
    esp_leaf_y = tree['EcalScoringPlaneHits_signal.y_'].array(library='np')
    esp_leaf_z = tree['EcalScoringPlaneHits_signal.z_'].array(library='np')
    esp_leaf_px = tree['EcalScoringPlaneHits_signal.px_'].array(library='np')
    esp_leaf_py = tree['EcalScoringPlaneHits_signal.py_'].array(library='np')
    esp_leaf_pz = tree['EcalScoringPlaneHits_signal.pz_'].array(library='np')

In [108]:
# Open the ROOT file and load branches
with uproot.open(file_path)['LDMX_Events'] as tree:
    tsp_leaf_pdgID = tree['TargetScoringPlaneHits_signal.pdgID_'].array(library='np')
    tsp_leaf_x = tree['TargetScoringPlaneHits_signal.x_'].array(library='np')
    tsp_leaf_y = tree['TargetScoringPlaneHits_signal.y_'].array(library='np')
    tsp_leaf_z = tree['TargetScoringPlaneHits_signal.z_'].array(library='np')
    tsp_leaf_px = tree['TargetScoringPlaneHits_signal.px_'].array(library='np')
    tsp_leaf_py = tree['TargetScoringPlaneHits_signal.py_'].array(library='np')
    tsp_leaf_pz = tree['TargetScoringPlaneHits_signal.pz_'].array(library='np')

In [109]:
# # Open the ROOT file and load branches
# with uproot.open(file_path) as file:
#     tree = file['skimmed_events']
#     leaf_pdgID = tree['pdgID_tsp_'].array(library='np')
#     leaf_x = tree['x_tsp_'].array(library='np')
#     leaf_y = tree['y_tsp_'].array(library='np')
#     leaf_z = tree['z_tsp_'].array(library='np')
#     leaf_px = tree['px_tsp_'].array(library='np')
#     leaf_py = tree['py_tsp_'].array(library='np')
#     leaf_pz = tree['pz_tsp_'].array(library='np')

### Apply check

In [110]:
print('ECal Scoring Plane')
esp_recoil_e_test = skim_esp_events(esp_leaf_pdgID, esp_leaf_z, esp_leaf_pz)
esp_oncell_test = check_projection(esp_leaf_x, esp_leaf_y, esp_leaf_z, esp_leaf_px, esp_leaf_py, esp_leaf_pz, esp_recoil_e_test, cells)

ECal Scoring Plane
total events: 20000
- no-electron events: 753
- invalid-z events: 9
- invalid-pz events: 0
+ valid events: 19238
validation: True
oncell recoil electron: 18628
offcell recoil electron: 610
offcell to oncell ratio: 0.0327464032639038


In [111]:
print('Target Scoring Plane')
tsp_recoil_e_test = skim_tsp_events(tsp_leaf_pdgID, tsp_leaf_z, tsp_leaf_pz)
tsp_oncell_test = check_projection(tsp_leaf_x, tsp_leaf_y, tsp_leaf_z, tsp_leaf_px, tsp_leaf_py, tsp_leaf_pz, tsp_recoil_e_test, cells)

Target Scoring Plane
total events: 20000
- no-electron events: 0
- invalid-z events: 7
- invalid-pz events: 0
+ valid events: 19993
validation: True
oncell recoil electron: 19959
offcell recoil electron: 34
offcell to oncell ratio: 0.001703492158925798


### Test codes

In [ ]:
# obsolete projection checker
def obs_check_projection(recoilX, recoilY, recoilZ, recoilPx, recoilPy, recoilPz, recoilElectron, cells):
    N = len(recoilElectron)
    oncell_cut = np.zeros(N, dtype=bool)
    # Loop through all events for projection check.
    for i in range(N):
        recoil_i = recoilElectron[i]
        # Skip invalid events, which contains no recoil electron.
        if recoil_i == -1:
            continue
        # Project to ECal face.
        fXY = projection(recoilX[i][recoil_i], recoilY[i][recoil_i], recoilZ[i][recoil_i], 
                         recoilPx[i][recoil_i], recoilPy[i][recoil_i], recoilPz[i][recoil_i], ecalFaceZ)
        # Loop through all cells for oncell check.
        for cell in cells:
            if dist(cell, fXY) <= cell_radius:
                oncell_cut[i] = 1
                break

    # Print results.
    oncell_events = len(oncell_test[np.where(oncell_test==1)])
    offcell_events = len(recoilElectron[np.where(recoilElectron != -1)])-oncell_events
    off_on_ratio = offcell_events/oncell_events
    print('oncell recoil electron: {}'.format(oncell_events))
    print('offcell recoil electron: {}'.format(offcell_events))
    print('offcell to oncell ratio: {}'.format(off_on_ratio))
    
    return oncell_cut

In [14]:
# Apply the fiducial cut
f_cut = apply_fiducial_cut(leaf_x, leaf_y, leaf_z, leaf_px, leaf_py, leaf_pz, recoil_electrons, cells)

In [15]:
# Calculate and print statistics
fiducial_events = np.sum(f_cut)
valid_events = np.count_nonzero(recoil_electrons != -1)
total_events = len(f_cut)
non_fid_events = valid_events-fiducial_events
ratio = non_fid_events / fiducial_events
print(f"Total Events: {total_events}")
print(f"Electron Events: {valid_events}")
print(f"Fiducial Events: {fiducial_events}")
print(f"Non-Fiducial Events: {non_fid_events}")
print(f"Non-Fiducial/Fiducial Events Ratio: {ratio:.4f}")

Total Events: 20000
Electron Events: 20000
Fiducial Events: 19998
Non-Fiducial Events: 2
Non-Fiducial/Fiducial Events Ratio: 0.0001


In [16]:
# skim_tsp_events_aw(leaf_pdgID, leaf_x, leaf_y, leaf_z, leaf_px, leaf_py, leaf_pz)

In [17]:
def skim_tsp_events_aw(pdgID, x, y, z, px, py, pz):
    # Masks for electrons (pdgID == 11)
    electron_mask = pdgID == 11

    # Max pz among electrons, default to -9999 where no electrons
    max_pz = ak.max(ak.where(electron_mask, pz, -9999), axis=1)

    # Indices of maximum pz
    max_pz_indices = ak.argmax(ak.where(electron_mask, pz, -9999), axis=1)

    # Conditions based on max pz
    negative_z = ak.any((z[max_pz_indices] < 0) & (pdgID[max_pz_indices] == 11), axis=1)
    pz_minus9999 = max_pz == -9999
    pz_zero = max_pz == 0

    # Valid pz (positive and not -9999 or zero)
    valid_pz_mask = (max_pz > 0) & (~pz_minus9999) & (~pz_zero)

    # Counters
    no_electrons = ak.sum(~ak.any(electron_mask, axis=1))
    negative_z_count = ak.sum(negative_z)
    pz_minus9999_count = ak.sum(pz_minus9999)
    pz_zero_count = ak.sum(pz_zero)
    valid_electron_indices = max_pz_indices[valid_pz_mask]

    return {
        'valid_electron_indices': valid_electron_indices,
        'no_electron_count': no_electrons,
        'negative_z_count': negative_z_count,
        'pz_minus9999_count': pz_minus9999_count,
        'pz_zero_count': pz_zero_count
    }

In [18]:
# # Plotting the histogram of fiducial vs non-fiducial events
# plt.hist(f_cut.astype(int), bins=[-0.5, 0.5, 1.5], rwidth=0.8) # label=['Non-Fiducial', 'Fiducial'])
# plt.xticks([0, 1], ['Non-Fiducial', 'Fiducial'])
# plt.xlabel('Event Type')
# plt.ylabel('Number of Events')
# plt.title('Fiducial vs Non-Fiducial Events')
# # plt.legend()
# plt.show()

In [42]:
# Calculate indices of recoil electron for all events
def find_recoil_electron(pdgID, pz):
    # Initialize indices as -1 (not found or conditions not met)
    r_electron = np.full(len(pdgID), -1, dtype=int)
    
    # Iterate over each event
    for i in range(len(pdgID)):
        # Find indices where particle ID is 11 (electron)
        electron_indices = np.where(pdgID[i] == 11)[0]
        
        # If no electrons or all pz values are zero, continue
        if len(electron_indices) == 0 or np.all(pz[i][electron_indices] == 0):
            continue
        
        # Find the index with the maximum pz among electrons
        # This step filters out all pz == 0 automatically since they can't be the maximum if there's any non-zero pz
        max_pz_index = electron_indices[np.argmax(pz[i][electron_indices])]
        
        # Set the recoil electron index for this event
        r_electron[i] = max_pz_index
    
    return r_electron
    
# Apply the fiducial cut to the given recoil data.
def apply_fiducial_cut(recoilX, recoilY, recoilZ, recoilPx, recoilPy, recoilPz, recoilElectron, cells):
    N = len(recoilElectron)
    f_cut = np.zeros(N, dtype=bool)
    for i in range(N):
        fiducial = False
        recoil_i = recoilElectron[i]
        if recoil_i != -1:
            fXY = projection(recoilX[i][recoil_i], recoilY[i][recoil_i], recoilZ[i][recoil_i], 
                             recoilPx[i][recoil_i], recoilPy[i][recoil_i], recoilPz[i][recoil_i], ecalFaceZ)
            # if not all(val == -9999 for val in [recoilX[i][recoil_i], recoilY[i][recoil_i], recoilZ[i][recoil_i], 
            #                                     recoilPx[i][recoil_i], recoilPy[i][recoil_i], recoilPz[i][recoil_i]]):
            for cell in cells:
                if dist(cell, fXY) <= cell_radius:
                    fiducial = True
                    # print(cell)
                    break
        f_cut[i] = fiducial
    return f_cut

## Recoil Electron

In [112]:
# Open the ROOT file and load branches
with uproot.open(file_path)['LDMX_Events'] as tree:
    veto_leaf_x = tree['recoilPz_'].array(library='np')

In [113]:
veto_recoil = np.where(veto_leaf_x == -9999)[0]

In [114]:
len(veto_recoil)

835

In [115]:
veto_leaf_x[68]

-9999.0

## Sth else

In [23]:
type(np.array([[1,2],[2,1],[3,1]]))

numpy.ndarray

In [24]:
np.where(np.array([11,2,3,11,4,5,-11]) == 11)[0]

array([0, 3])

In [25]:
arr = np.array([1,2,3,4,5,6,7])

In [26]:
np.argmax(arr[[1,2,3]])

2